In [5]:
import pretty_midi
import numpy as np
import glob
import mido
import os
import tensorflow as tf
import random

notes_array = np.load('/Users/rich/Desktop/AI-music/NumpyArray/notes_array.npy')
velocities_array = np.load('/Users/rich/Desktop/AI-music/NumpyArray/velocities_array.npy')

#normalize
#帶來以下好處:
#1. 處理不同尺度的數據
#2. 加快模型收斂速度

#change shape
notes_normalized = notes_array.reshape(-1, 1) / int(notes_array.max())
velocities_normalized = velocities_array.reshape(-1, 1) / float(velocities_array.max())

# combine notes and velocities
combined_data = np.concatenate((notes_normalized, velocities_normalized), axis=1)
combined_data.shape

(2599480, 2)

In [11]:
import torch
import torch.nn as nn
from transformers import BertModel, GPT2Model

# 模型定义
class BERTMusicGenerator(nn.Module):
    def __init__(self):
        super().__init__()
        self.bert = BertModel.from_pretrained("bert-base-uncased")
        self.linear = nn.Linear(768, 2)

    def forward(self, x):
        x = self.bert(x)
        x = x.last_hidden_state
        x = x[:, 0, :]
        x = self.linear(x)
        return x

class GPTMusicGenerator(nn.Module):
    def __init__(self):
        super().__init__()
        self.gpt = GPT2Model.from_pretrained("gpt2")
        self.linear = nn.Linear(768, 2)  # GPT2Model的输出维度为768

    def forward(self, x):
        x = self.gpt(x)
        x = x.last_hidden_state
        x = x[:, 0, :]
        x = self.linear(x)
        return x

# 模型训练
bert_model = BERTMusicGenerator()
gpt_model = GPTMusicGenerator()

optimizer_bert = torch.optim.Adam(bert_model.parameters(), lr=0.001)
optimizer_gpt = torch.optim.Adam(gpt_model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

# 虚拟数据集（使用 combined_data）
combined_data = torch.tensor(combined_data, dtype=torch.float32)

for epoch in range(10):
    for batch in combined_data:
        # BERT
        optimizer_bert.zero_grad()
        bert_outputs = bert_model(batch.unsqueeze(0))
        bert_loss = criterion(bert_outputs, torch.tensor([1]))  # 假设标签为1
        bert_loss.backward()
        optimizer_bert.step()

        # GPT
        optimizer_gpt.zero_grad()
        gpt_outputs = gpt_model(batch.unsqueeze(0))
        gpt_loss = criterion(gpt_outputs, torch.tensor([1]))  # 假设标签为1
        gpt_loss.backward()
        optimizer_gpt.step()

        # 損失函數
        loss = bert_loss + gpt_loss

        # 打印训练信息
        print(f"Epoch {epoch + 1}, Loss: {loss.item()}")

# 模型保存
torch.save(bert_model.state_dict(), "bert_model.pth")
torch.save(gpt_model.state_dict(), "gpt_model.pth")


RuntimeError: Expected tensor for argument #1 'indices' to have one of the following scalar types: Long, Int; but got torch.FloatTensor instead (while checking arguments for embedding)